In [1]:
# load modules to view the bonds we need to move

In [1]:
from ase.io import read, write
from ase.visualize import view
from ase import Atoms
import os
import numpy as np
import random

In [2]:
frame = read('BaseFrame.xyz', ':')
baseframe = frame[0]
baseframe.wrap(eps=1e-15)

In [3]:
view(baseframe, viewer='ngl')

In [6]:
si_atoms = []
for i in range(len(baseframe)):
    if baseframe.symbols[i] == 'Si':
        si_atoms.append(i)

In [15]:
rand_si = si_atoms[random.randrange(len(si_atoms))]
print(rand_si)

106


In [18]:
# get near atoms
cell = [100, 100, 100]
positions_l = []
frame_distances = baseframe.get_all_distances()[rand_si]
for i in range(len(frame_distances)):
    if frame_distances[i] <= 5:
        symbol = baseframe.symbols[i]
        positions = baseframe.positions[i]
        positions_l.append(list(positions))
        print(f'{symbol} : {positions} : index {i}')
        
atoms = Atoms('SiO4', positions_l)

Si : [ 8.99131678 34.5290753  14.7341944 ] : index 106
O : [ 6.24176531 33.79208212 13.55122586] : index 119
O : [ 8.73620376 34.76340134 17.75397671] : index 246
O : [ 9.95696682 37.31075212 13.40949641] : index 250
O : [11.17017098 32.3351033  14.03499574] : index 252


In [19]:
view(atoms, viewer='ngl')

In [23]:
Si = np.array([8.99131678, 34.5290753,  14.7341944])
Oxy = np.array([8.73620376, 34.76340134, 17.75397671])

origin = np.array([0, 0, 0])

vector_d = Oxy - Si
vector_d 
sum_d = 0
for i in range(vector_d.shape[0]):
    sum_d += float(vector_d[i])**2
    
mag = np.sqrt(sum_d)
print(vector_d, mag)

[-0.25511302  0.23432604  3.01978231] 3.039584929852189


In [26]:
b_OO = 3.039584929852189
b_O = 3.039584929852189
list_bs = []
array_BO = np.zeros([10,4])
for i in range(10):
    b_O -= 0.2039584929852189
    list_bs.append(b_O)
    vector_N = vector_d * (b_O/b_OO)
    print(f'{b_O}: {vector_N}')
    array_BO[i,0] = b_O
    array_BO[i,1] = vector_N[0]
    array_BO[i,2] = vector_N[1]
    array_BO[i,3] = vector_N[2]

array_BO
np.save('Si-O_vectors.npy', array_BO)

2.83562643686697: [-0.23799474  0.21860258  2.81715259]
2.631667943881751: [-0.22087646  0.20287912  2.61452287]
2.4277094508965322: [-0.20375818  0.18715567  2.41189314]
2.2237509579113133: [-0.1866399   0.17143221  2.20926342]
2.0197924649260943: [-0.16952162  0.15570875  2.0066337 ]
1.8158339719408754: [-0.15240334  0.13998529  1.80400398]
1.6118754789556564: [-0.13528506  0.12426183  1.60137426]
1.4079169859704375: [-0.11816678  0.10853838  1.39874453]
1.2039584929852185: [-0.1010485   0.09281492  1.19611481]
0.9999999999999996: [-0.08393022  0.07709146  0.99348509]


In [28]:
# compute the new oxygen position vector as Cartesian coordinates
Oxy_array = np.zeros([10,3])
for i in range(array_BO.shape[0]):
    Oxy_array[i,:] = Si + array_BO[i,1:]
    
Oxy_array
np.save('O_positions.npy', Oxy_array)

In [31]:
for i in range(Oxy_array.shape[0]):
    new_frame = baseframe
    positions = new_frame.positions
    positions[246] = Oxy_array[i,:]
    string = f'Si-O_{round(array_BO[i,0], 5)}.xyz'
    write(string, new_frame)